In [3]:
import pymysql
class ConnectDatabase:
    def connect(self,host,user,passwd,database):
        self.db=pymysql.connect(host=host,user=user,password=passwd,database=database)
        self.cursor=self.db.cursor()
    def close(self):
        self.db.close()
        self.cursor.close()
    def executeSql(self,sql):
        self.cursor.execute(sql)
        return self.cursor.fetchall()

In [4]:
import pyecharts.options as opts
from pyecharts.charts import Bar
from pyecharts.commons.utils import JsCode
from pyecharts.globals import ThemeType
from pyecharts.charts import Line
from pyecharts.charts import PictorialBar
from pyecharts.globals import SymbolType
from pyecharts.components import Table
from pyecharts.options import ComponentTitleOpts
class Visible:
    #实现柱状图
    def Bar(self,x,y1,y2,name1,name2,label,name):
        c = (
            Bar(init_opts=opts.InitOpts(theme=ThemeType.LIGHT))
            .add_xaxis(x)
            .add_yaxis(name1, y1, stack="stack1", category_gap="50%")
            .add_yaxis(name2, y2,stack="stack1", category_gap="50%")
            .set_series_opts(
                label_opts=opts.LabelOpts(
                    position="right",
                    formatter=JsCode(
                        "function(x){return Number(x.data.percent * 100).toFixed() + '%';}"
                    ),
                )
            )
        )
        if label==0:
            return c.render_notebook()
        elif label==-1:
                return c
        else:
            return c.render(name)
    def Line(self,x,y,label,name):
            line=(
                Line()
                .set_global_opts(
                    tooltip_opts=opts.TooltipOpts(is_show=False),
                    xaxis_opts=opts.AxisOpts(type_="category"),
                    yaxis_opts=opts.AxisOpts(
                        type_="value",
                        axistick_opts=opts.AxisTickOpts(is_show=True),
                        splitline_opts=opts.SplitLineOpts(is_show=True),
                    ),
                )
                .add_xaxis(xaxis_data=x)
                .add_yaxis(
                    series_name="评分折线图",
                    y_axis=y,
                    symbol="emptyCircle",
                    is_symbol_show=True,
                    label_opts=opts.LabelOpts(is_show=False),
                )
            )
            if label==0:
                return line.render_notebook()
            elif label==-1:
                return line
            else:
                return line.render(name)
    def elephantBar(self,x,y,label,name):
        c = (
            PictorialBar()
            .add_xaxis(x)
            .add_yaxis(
                "",
                y,
                label_opts=opts.LabelOpts(is_show=False),
                symbol_size=18,
                symbol_repeat="fixed",
                symbol_offset=[0, 0],
                is_symbol_clip=True,
                symbol=SymbolType.ROUND_RECT,
            )
            .reversal_axis()
            .set_global_opts(
                title_opts=opts.TitleOpts(title="每年电影的总观看量"),
                xaxis_opts=opts.AxisOpts(is_show=False),
                yaxis_opts=opts.AxisOpts(
                    axistick_opts=opts.AxisTickOpts(is_show=False),
                    axisline_opts=opts.AxisLineOpts(
                        linestyle_opts=opts.LineStyleOpts(opacity=0)
                    ),
                ),
            )
        )
        if label==0:
            return c.render_notebook()
        elif label==-1:
                return c
        else:
            return c.render(name)
    def table(self,head,rows,label,name):
        table = Table()
        headers = head
        table.add(headers, rows)
        table.set_global_opts(
            title_opts=ComponentTitleOpts(title="每年上映的电影数量,平均评分和总观看人数", subtitle="副标题")
        )
        if label==0:
            return table.render_notebook()
        elif label==-1:
                return table
        else:
            return table.render(name)

In [5]:
#数据预处理1
con=ConnectDatabase()
con.connect("localhost","root","root","data")
sql='select year,count(*) from dt3 where score>5 group by year order by year;'
result=con.executeSql(sql)
year=list(range(2010,2022))
big=[]
for i in range(len(year)):
    big.append({"value":0})
for row in result:
    big[year.index(row[0])]['value']=row[1]
sql='select year,count(*) from dt3 where score<=5 group by year order by year;'
result=con.executeSql(sql)
little=[]
for i in range(len(year)):
    little.append({"value":0})
for row in result:
    little[year.index(row[0])]['value']=row[1]
print(len(big),len(little))
for row in range(len(big)):
    big[row]["percent"]=big[row]["value"]/(big[row]["value"]+little[row]["value"])
    little[row]["percent"]=little[row]["value"]/(big[row]["value"]+little[row]["value"])
newYear=[]
for row in year:
    newYear.append(str(row))

12 12


In [6]:
visible=Visible()
visible.Bar(newYear,big,little,"评分大于5的电影数量","评分小于5的电影数量",1,'1.html')

'C:\\Users\\Lenovo\\hadoop实训\\1.html'

In [7]:
#数据处理2
sql="select year,avg(score) from dt3 group by year;"
result=con.executeSql(sql)
year=[]
avgscore=[]
for row in result:
    year.append(str(row[0]))
    avgscore.append(round(row[1],2))

In [8]:
visible.Line(year,avgscore,1,'2.html')

'C:\\Users\\Lenovo\\hadoop实训\\2.html'

In [9]:
#数据处理3
sql="select year,sum(coun) coun from dt3 group by year;"
result=con.executeSql(sql)
year=[]
visit=[]
for row in result:
    year.append(str(row[0]))
    visit.append(int(row[1]))

In [10]:
visible.elephantBar(year,visit,1,'3.html')

'C:\\Users\\Lenovo\\hadoop实训\\3.html'

In [11]:
#数据预处理4
sql="select year,count(*),avg(score),sum(coun) from dt3 group by year;"
result=con.executeSql(sql)
head=["年份","电影数量","平均评分","总观看人数"]
rows=[]
for row in result:
    rows.append([row[0],row[1],round(row[2],2),row[3]])

In [12]:
visible.table(head,rows,1,'4.html')

'C:\\Users\\Lenovo\\hadoop实训\\4.html'

In [17]:
import json
import pandas as pd
import numpy as np
from pyecharts import options as opts 
from pyecharts.charts import Geo,Map,Bar, Line, Page,Pie, Boxplot, WordCloud
from pyecharts.globals import ChartType, SymbolType,ThemeType
page = Page(layout=Page.DraggablePageLayout)
page.add(visible.Bar(newYear,big,little,"评分大于5的电影数量","评分小于5的电影数量",-1,'1.html'),visible.Line(year,avgscore,-1,'2.html')
        ,visible.elephantBar(year,visit,-1,'3.html'),visible.table(head,rows,-1,'4.html'))
#page.render("test.html")
Page.save_resize_html("test.html", cfg_file="chart_config.json",dest="my_test.html")

'<!DOCTYPE html>\n<html>\n<head>\n    <meta charset="UTF-8">\n    <title>Awesome-pyecharts</title>\n            <script type="text/javascript" src="https://assets.pyecharts.org/assets/echarts.min.js"></script>\n        <script type="text/javascript" src="https://assets.pyecharts.org/assets/jquery.min.js"></script>\n        <script type="text/javascript" src="https://assets.pyecharts.org/assets/jquery-ui.min.js"></script>\n        <script type="text/javascript" src="https://assets.pyecharts.org/assets/ResizeSensor.js"></script>\n\n            <link rel="stylesheet"  href="https://assets.pyecharts.org/assets/jquery-ui.css">\n\n</head>\n<body>\n    <style>.box {  }; </style>\n        \n    <div class="box">\n                <div id="7afcb7aa24604a4a811a2f00c20e8798" class="chart-container" style="position: absolute; width: 523px; height: 330px; top: 31.80000114440918px; left: 501px;"></div>\n    <script>\n        var chart_7afcb7aa24604a4a811a2f00c20e8798 = echarts.init(\n            docu

In [19]:
words = [
    ("Sam S Club", 10000),
    ("Macys", 6181),
    ("Amy Schumer", 4386),
    ("Jurassic World", 4055),
    ("Charter Communications", 2467),
    ("Chick Fil A", 2244),
    ("Planet Fitness", 1868),
    ("Pitch Perfect", 1484),
    ("Express", 1112),
    ("Home", 865),
    ("Johnny Depp", 847),
    ("Lena Dunham", 582),
    ("Lewis Hamilton", 555),
    ("KXAN", 550),
    ("Mary Ellen Mark", 462),
    ("Farrah Abraham", 366),
    ("Rita Ora", 360),
    ("Serena Williams", 282),
    ("NCAA baseball tournament", 273),
    ("Point Break", 265),
]
c = (
    WordCloud()
    .add("", words, word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title="WordCloud-shape-diamond"))
    .render_notebook()
)

In [32]:
sql="select title,coun from dt3"
result=con.executeSql(sql)
l=[]
for row in result:
    l.append([str(row[0]),int(row[1])])

In [33]:
l

[['觉醒年代', 32183],
 ['我买了一个农场', 4892],
 ['大人物', 99563],
 ['诸神的黄昏 第二季', 29691],
 ['洛基 第一季', 73439],
 ['致命女人 第二季', 85808],
 ['东城梦魇', 18419],
 ['以你的心诠释我的爱 第二季', 30560],
 ['御赐小仵作', 73517],
 ['Move to Heaven：我是遗物整理师', 84004],
 ['月光变奏曲', 55445],
 ['心惊胆战的同居', 81779],
 ['时光代理人', 81279],
 ['爱上特种兵', 17840],
 ['光荣与梦想', 54074],
 ['大豆田永久子与三名前夫', 38326],
 ['我的青铜时代', 61527],
 ['顶楼 第三季', 30755],
 ['眷思量', 98441],
 ['以你的心诠释我的爱', 80027],
 ['9号秘事 第六季', 33079],
 ['山海情', 67196],
 ['庆余年 第一季', 8466],
 ['鹿角男孩', 59430],
 ['山河令', 92834],
 ['萌探探探案', 13040],
 ['无罪之最', 37757],
 ['短剧开始啦', 60286],
 ['窥探', 82327],
 ['某天，灭亡从我家玄关进来了', 13542],
 ['向往的生活 第五季', 6048],
 ['逆天奇案', 56683],
 ['五十公里桃花坞', 38256],
 ['禁忌女孩', 80481],
 ['至上之法', 91182],
 ['放学别走', 31052],
 ['致不灭的你', 62449],
 ['双世宠妃Ⅲ', 73802],
 ['理想照耀中国', 42121],
 ['春季露营', 48031],
 ['火神的眼泪', 37472],
 ['沉默的真相', 32462],
 ['猎狼者', 312],
 ['禁忌女孩2', 30956],
 ['后翼弃兵', 44839],
 ['爱，死亡和机器人 第二季', 64561],
 ['四重奏', 56594],
 ['请回答1988', 35657],
 ['五月的青春', 25395],
 ['你是我的城池营垒', 29824],

In [35]:
words = l
c = (
    WordCloud()
    .add("", words, word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    .set_global_opts(title_opts=opts.TitleOpts(title="WordCloud-shape-diamond"))
    .render('myciyun.html')
)